# FINAL PROJECT SUBMISSION

- Student Name: Ian Korir
- Student Pase: Fulltime-Hybrid student
- Scheduled Project Review: 22/03/2024
- Instructor Name: Nikita Njoroge
- Blog Post: 

# Introduction
## Overview
Business Problem:
Viusasa new movie studio aims to establish itself as a leading player in the entertainment industry by producing high-quality and commercially successful movies. However, the studio faces the challenge of navigating the complexities of the movie market to ensure its productions resonate with audiences and generate significant revenue.


Problem Statement: How can Viusasa's new movie studio strategically optimize its production and release strategy to maximize box office success and audience satisfaction?

## Objectives
Key Objectives:
1.	Identify the most profitable and audience-friendly movie genres to prioritize in production efforts.
2.	Determine the optimal timing for movie releases to maximize audience turnout and box office revenue.
3.	Enhance audience engagement and satisfaction by aligning movie offerings with audience preferences and expectations.

## The Data
The datasets are stored in the 'zippedData' folder which were sourced from:
1.	Box Office MojoLinks : 
	bom.movie_gross.csv
2.	IMDBLinks : 
	im.db
3.	Rotten TomatoesLinks : 
	rt.movie_info.tsv
4.	TheMovieDBLinks : 
	tmdb.movies.csv
5.	The Numbers :
	tn.movie_budgets.csv




## Import relevant libraries


In [ ]:
#importing libraries
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt